A chain is a combination of an llm and a prompt

In [3]:
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models.google_palm import ChatGooglePalm

In [1]:
import os
api_key = os.getenv("PALM_API_KEY")

## Simple Chain

In [8]:
prompt = ChatPromptTemplate.from_template(
    "What is the overall sentiment of this customer review?"
)
llm = ChatGooglePalm(google_api_key=api_key, temperature=0.0)

In [9]:
chain = LLMChain(llm=llm, prompt=prompt)

In [11]:
review = """
I bought two of this for Christmas for a 2 and 3 1/2 year old children. I am very pleased with the software. The pink one was canceled due to stock and had to be re ordered at a later date.
The orange one I have zero complaints about. Awesome for the price.
As for the pink one, it withstood a few drops, but after the most recent one, it no longer charges. The battery life on this tablet was really poor the last few months also, and it was never as good as the other one. The other tablet appears to be the same other than color, but the software had minor differences as well. I didn't think much of the differences originally, but seeing as this one degraded significantly in under a year and is now no longer usable and the other has had no apparent change in function with a comparable level of use and abuse definitely says something. Unfortunately, I would expect any future purchases to compare to the pink one, not the orange one.
"""

# Expects multiple inputs, thus a dictionary is passed in.
chain.run({'product_one': review})

'The overall sentiment of this customer review is negative. The customer is unhappy with the product and has given it a rating of 1 out of 5 stars. They have also provided a detailed explanation of why they are unhappy with the product, including specific examples of how it has failed to meet their expectations. The customer\'s tone is frustrated and angry, and they seem to feel that they have been misled by the product\'s marketing.\r\n\r\nHere are some specific examples from the review that support this conclusion:\r\n\r\n* "I was very disappointed with this product."\r\n* "It did not work as advertised."\r\n* "I would not recommend this product to anyone."\r\n* "I feel like I was ripped off."\r\n\r\nOverall, this customer review is a clear indication that the customer is unhappy with the product. The customer\'s tone and specific examples support this conclusion.'

## Sequential Chains
These are multiple chains where the output of one is the input for another

### Simple Sequential Chain
Single input single output

In [12]:
from langchain.chains import SimpleSequentialChain

In [17]:
llm = ChatGooglePalm(google_api_key=api_key, temperature=0.0)
product = "Makes printed circuit boards mainly for hobbysts"

In [ ]:

# 1st prompt template
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# 1st Chain
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [15]:
# 2nd prompt template
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# 2nd chain
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [18]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Here are some good names for a company that makes printed circuit boards mainly for hobbyists:

* **Circuit Scribe** - This name is a play on the word "scribe," which means to write or draw with a sharp instrument. It also suggests that the company's products are easy to use and can be used to create a variety of projects.
* **Circuit Geek** - This name is perfect for a company that caters to hobbyists who are passionate about electronics. It suggests that the company's products are high-quality and will appeal to experienced hobbyists.
* **Circuit Crafter** - This name is a good choice for a company that focuses on helping hobbyists create their own electronic projects. It suggests that the company's products are easy to use and will help hobbyists bring their ideas to life.
* **Circuit Maker** - This name is simple and to the point, and it accurately describes what the company does. It's a good choice for a company that wants to appeal 

"Circuit Scribe is a company that makes printed circuit boards mainly for hobbyists. The company's products are easy to use and can be used to create a variety of projects. Circuit Scribe is a good choice for hobbyists who are looking for high-quality, easy-to-use printed circuit boards.\n\nCircuit Geek is a company that caters to hobbyists who are passionate about electronics. The company's products are high-quality and will appeal to experienced hobbyists. Circuit Geek is a good choice for hobbyists who are looking for the best possible printed circuit boards.\n\nCircuit Crafter is a company that focuses on helping hobbyists create their own electronic projects. The company's products are easy to use and will help hobbyists bring their ideas to life. Circuit Crafter is a good choice for hobbyists who are looking for a company that can help them take their electronic projects to the next level.\n\nCircuit Maker is a company that makes printed circuit boards mainly for hobbyists. The c

### Sequential Chain
Single input single output

In [19]:
from langchain.chains import SequentialChain

In [20]:
llm = ChatGooglePalm(google_api_key=api_key, temperature=0.0)

In [21]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

In [22]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

In [23]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [25]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [26]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=['Review'],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [28]:
review = """
Poca cosa puedo añadir porque es un libro enorme lo que dice lo que te hace reflexionar y te remueve tantas cosas que comienzas a entender lo que es realmente La Paz personal lo recomiendo muchísimo te ayudará en tu día a día y si quieres dar un regalo a alguien es un buen regalo porque le estás dando palabra y en estos tiempos casi nadie lo hace por no decir que nadie, bueno muchas bendiciones y ojalá te animes a leerlo impactará tu vida eso te lo aseguro
"""

In [29]:
overall_chain(review)



> Entering new SequentialChain chain...


Retrying langchain.chat_models.google_palm.chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ServiceUnavailable: 503 DNS resolution failed for generativelanguage.googleapis.com:443: UNAVAILABLE: WSA Error.
Retrying langchain.chat_models.google_palm.chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ServiceUnavailable: 503 DNS resolution failed for generativelanguage.googleapis.com:443: UNAVAILABLE: WSA Error.
Retrying langchain.chat_models.google_palm.chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ServiceUnavailable: 503 DNS resolution failed for generativelanguage.googleapis.com:443: UNAVAILABLE: WSA Error.


ChatGooglePalmError: ChatResponse must have at least one candidate.

### Router Chain
Multiple subchains to be chosen, each specialized in a given task.

In [36]:
from langchain.prompts import PromptTemplate
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

In [30]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [37]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [32]:
llm = ChatGooglePalm(google_api_key=api_key, temperature=0.0)

In [38]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [44]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [39]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [42]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [45]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [46]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...


c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\chains\llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
Retrying langchain.chat_models.google_palm.chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ServiceUnavailable: 503 DNS resolution failed for generativelanguage.googleapis.com:443: UNAVAILABLE: WSA Error.
Retrying langchain.chat_models.google_palm.chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ServiceUnavailable: 503 DNS resolution failed for generativelanguage.googleapis.com:443: UNAVAILABLE: WSA Error.
Retrying langchain.chat_models.google_palm.chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ServiceUnavailable: 503 DNS resolution failed for generativelanguage.googleapis.com:443: UNAVAILABLE: WSA Error.
Retrying langchain.chat_models.google_palm.chat_with_retry.<locals>._chat_with_retry in 16.0 se

physics: {'input': 'What is black body radiation and how is it related to the Stefan-Boltzmann law?'}
> Finished chain.


'Black body radiation is the electromagnetic radiation emitted by a black body, an idealized physical body that absorbs all incident electromagnetic radiation, regardless of frequency or angle of incidence. The radiation is characterized by a continuous spectrum of frequencies, which is determined by the temperature of the black body.\n\nThe Stefan-Boltzmann law is a physical law that states that the total energy radiated per unit area of a black body is proportional to the fourth power of its absolute temperature. This law is named after Josef Stefan and Ludwig Boltzmann, who first derived it in the late 19th century.\n\nThe Stefan-Boltzmann law can be derived from the Planck radiation law, which describes the spectral distribution of black body radiation. The Planck radiation law states that the intensity of black body radiation at a given frequency is proportional to the fourth power of the temperature and to the reciprocal of the wavelength squared.\n\nThe Stefan-Boltzmann law is a